In [6]:
from joblib import load
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

final_model_pipeline = load('../modeling/best_models/StackingClassifier/model.joblib')
final_model = final_model_pipeline.named_steps['model']
processing_pipeline = final_model_pipeline.named_steps['processing']
engineering_pipeline = final_model_pipeline.named_steps['engineering']

In [7]:
X_train = pd.read_csv('../data/orig_train.csv')

y_train = X_train['DEFAULT_FLAG']
X_train.drop(columns=['DEFAULT_FLAG'], inplace=True)

processing_pipeline.fit(X_train)
X_train = processing_pipeline.transform(X_train)
engineering_pipeline.fit(X_train)
X_train = engineering_pipeline.transform(X_train)

In [13]:
X_val = pd.read_csv('../data/orig_valid.csv')

y_val = X_val['DEFAULT_FLAG']
X_val.drop(columns=['DEFAULT_FLAG'], inplace=True)

X_val = processing_pipeline.transform(X_val)
X_val = engineering_pipeline.transform(X_val)

In [8]:
import shap

In [14]:
final_model.fit(X_train, y_train)
y_pred = final_model.predict(X_val)

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_val)

In [46]:
explainer = shap.Explainer(clf)
shap_values = explainer.shap_values(X_val)

In [69]:
from pdpbox import pdp

eduaction_features = sorted([c for c in X_val if c.startswith('EDUCATION_')])

pdp_cabin = pdp.PDPIsolate(
    model=final_model,
    df=pd.concat([X_val, y_val], axis=1),
    model_features=X_val.columns,
    feature=eduaction_features,
    feature_name='Education'
)

fig, axes = pdp_cabin.plot(
    center=True,
    plot_lines=False,
    plot_pts_dist=True,
    to_bins=False,
    engine='matplotlib',
)
fig.set_figheight(5)

ValueError: feature should be one-hot encoded